In [2]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 05
### Team Member Names: Piero Camposeo, Sathun Suthakaran, Ishaan Bansal
### Team Strategy Chosen: SAFE

*Abstract and explanation go here*

In [29]:
#Read in csv file of tickers and make values into a list
csv_file = "Tickers.csv"

tickers = pd.read_csv(csv_file)
ticker_list = tickers.iloc[:, 0].values.tolist()
ticker_list.insert(0, tickers.columns[0])

In [31]:
def filter_stocks(ticker_list):
    newlist = []

    for ticker in ticker_list:
        tick = yf.Ticker(ticker)

        try:
            if (tick.info['financialCurrency'] == 'USD'):
                newlist.append(ticker)
        except:
            print(f"Cannot get stock information of {ticker}. It may be delisted")

    return(newlist)
    
filter_stocks(ticker_list)



Cannot get stock information of AGN. It may be delisted
Cannot get stock information of CELG. It may be delisted
Cannot get stock information of PCLN. It may be delisted
Cannot get stock information of RTN. It may be delisted
Cannot get stock information of TWX. It may be delisted


['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CSCO',
 'CVS',
 'GM',
 'GOOG',
 'JPM',
 'KMI',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NKE',
 'ORCL',
 'OXY',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'SBUX',
 'SLB',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

In [ ]:
#Sathun

In [ ]:
#Ishaan

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Piero Camposeo, Sathun Suthakaran, Ishaan Bansal